# Libraries and Imports


In [ ]:
import torch
import torch.nn as nn
import torch.multiprocessing
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import DataLoader
from torchvision import transforms
from torchvision.io import read_image
import os
import pandas as pd
from tqdm import tqdm
from matplotlib import pyplot as plt
import random
import torchviz
from PIL import Image
from tqdm import tqdm

# Data and Processing


In [ ]:
class NumaGuardDataset(torch.utils.data.Dataset):
    def __init__(self, data_dir, transform=None):
        super(NumaGuardDataset, self).__init__()
        self.data_dir = data_dir
        self.transform = transform

        self.data = []
        for label in os.listdir(data_dir):
            label_dir = os.path.join(data_dir, label)
            for img in os.listdir(label_dir):
                self.data.append((os.path.join(label_dir, img), int(label)))

        print(f"Loaded {len(self.data)} images")

    def __getitem__(self, index):
        img_path, label = self.data[index]
        img = read_image(img_path)

        if self.transform:
            img = self.transform(img)
        img = img.float() / 255.0

        return img, label

    def __len__(self):
        return len(self.data)

In [ ]:
# check the dataset
dataset = NumaGuardDataset("data")
img, label = dataset[0]

plt.imshow(img.permute(1, 2, 0))
plt.title(f"Label: {label}")
plt.show()

# Model


In [ ]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(
            3, 16, 7, 2, 3
        ) 
        self.bn1 = nn.BatchNorm2d(16)
        self.conv2 = nn.Conv2d(16, 32, 5, 1, 2)  
        self.bn2 = nn.BatchNorm2d(32)
        self.conv3 = nn.Conv2d(32, 64, 3, 1, 1)
        self.bn3 = nn.BatchNorm2d(64)
        self.fc1 = nn.Linear(73728, 512) 
        self.fc2 = nn.Linear(512, 10)

    def forward(self, x):
        x = F.relu(self.bn1(self.conv1(x)))
        x = F.max_pool2d(x, 2, 2)
        x = F.relu(self.bn2(self.conv2(x)))
        x = F.max_pool2d(x, 2, 2)
        x = F.relu(self.bn3(self.conv3(x)))
        x = F.max_pool2d(x, 2, 2)
        x = torch.flatten(x, 1)
        x = F.relu(self.fc1(x))
        x = self.fc2(x) # no activation function since we are using CrossEntropyLoss
        return x

In [ ]:
params = {
    "batch_size": 64,
    "test_batch_size": 100,
    "epochs": 30,
    "lr": 0.001,
    "momentum": 0.9,
    "seed": 1,
    "log_interval": 10,
    "num_workers": 0,
    "pin_memory": True,
    "patience": 20, 
}

In [ ]:
def train(model, dataloader, criterion, optimizer, device, scaler):
    model.train()
    running_loss = 0.0

    for imgs, labels in tqdm(dataloader, desc="Training", leave=False):
        imgs = imgs.to(device, non_blocking=True)
        labels = labels.to(device, non_blocking=True)

        optimizer.zero_grad()

        with torch.cuda.amp.autocast():
            outputs = model(imgs)
            loss = criterion(outputs, labels)

        scaler.scale(loss).backward()
        scaler.step(optimizer)
        scaler.update()

        running_loss += loss.item()

    avg_loss = running_loss / len(dataloader)
    return avg_loss


def validate(model, dataloader, criterion, device):
    model.eval()
    total_loss = 0.0
    total_correct = 0

    with torch.no_grad():
        for imgs, labels in tqdm(dataloader, desc="Validation", leave=False):
            imgs = imgs.to(device, non_blocking=True)
            labels = labels.to(device, non_blocking=True)

            with torch.cuda.amp.autocast():
                outputs = model(imgs)
                loss = criterion(outputs, labels)

            total_loss += loss.item()
            total_correct += (
                (outputs.argmax(1) == labels).type(torch.float).sum().item()
            )

    accuracy = total_correct / len(dataloader.dataset)
    avg_loss = total_loss / len(dataloader)
    return avg_loss, accuracy

In [ ]:
transform = transforms.Compose(
    [
        transforms.Resize((196, 196), antialias=True),
        # transforms.RandomRotation(10),
        # transforms.ColorJitter(brightness=0.7, contrast=0.7, saturation=0.7, hue=0.5),
        # transforms.RandomPerspective(distortion_scale=0.3, p=0.5),
    ]
)
dataset: NumaGuardDataset = NumaGuardDataset(
    data_dir="./data",
    transform=transform,
)

train_size = int(0.7 * len(dataset))
val_size = (len(dataset) - train_size) // 2
test_size = len(dataset) - train_size - val_size

train_dataset, val_dataset, test_dataset = torch.utils.data.random_split(
    dataset,
    [train_size, val_size, test_size],
)

train_loader = DataLoader(
    train_dataset,
    batch_size=params["batch_size"],
    shuffle=True,
    pin_memory=True,
)
val_loader = DataLoader(
    val_dataset,
    batch_size=params["batch_size"],
    shuffle=False,
    pin_memory=True,
)
test_loader = DataLoader(
    test_dataset,
    batch_size=params["batch_size"],
    shuffle=False,
    pin_memory=True,
)

# Training


In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = Net().to(device)
criterion = nn.CrossEntropyLoss()
scaler = torch.cuda.amp.GradScaler()
# optimizer = optim.Adam(model.parameters(), lr=args["lr"])
optimizer = optim.SGD(model.parameters(), lr=params["lr"], momentum=params["momentum"])
# scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=5, gamma=0.2)

best_val_loss = float("inf")
best_model_weights = None
epochs_since_improvement = 0
train_losses = []
val_losses = []
val_accuracies = []

for epoch in range(1, params["epochs"] + 1):
    train_loss = train(model, train_loader, criterion, optimizer, device, scaler)
    val_loss, val_acc = validate(model, val_loader, criterion, device)
    # scheduler.step(val_loss)

    if val_loss < best_val_loss:
        best_val_loss = val_loss
        best_model_weights = model.state_dict()
        epochs_since_improvement = 0
    else:
        epochs_since_improvement += 1

    if epochs_since_improvement >= params["patience"]:
        print(f"Early stopping at epoch {epoch}")
        break

    train_losses.append(train_loss)
    val_losses.append(val_loss)
    val_accuracies.append(val_acc)

    print(f'Epoch:              {epoch}/{params["epochs"]}')
    print(f"Training Loss:      {train_loss:.4f}")
    print(
        f"Validation Loss:    {val_loss:.4f} | Accuracy: {val_acc:.4f}"
    )
    print(f'Learning rate: {optimizer.param_groups[0]["lr"]:.7f}')


In [ ]:

torch.save(best_model_weights, "numaguard_cnn.pt")

model.load_state_dict(best_model_weights)
test_loss, test_acc = validate(model, test_loader, criterion, device)

print(f"\nFinal Results:")
print(f"Test Loss: {test_loss:.4f}")
print(f"Test Accuracy: {test_acc:.2f}%")

# Evaluation


In [ ]:
# plot training & validation loss values
fig, ax = plt.subplots(1, 2, figsize=(10, 5))
ax[0].plot(train_losses, label="Training loss")
ax[0].plot(val_losses, label="Validation loss")
ax[0].set_xlabel("Epoch")
ax[0].set_ylabel("Loss")
# ax[0].set_yscale('log')
ax[0].legend()

# plot validation accuracy
ax[1].plot(val_accuracies, label="Validation accuracy")
ax[1].set_xlabel("Epoch")
ax[1].set_ylabel("Accuracy")
ax[1].legend()

# make it fit better
plt.tight_layout()